In [1]:
from discovery_utils.getters import gtr
from discovery_utils.getters import crunchbase
from discovery_utils.utils import search

from src import PROJECT_DIR
from src import VECTOR_DB_DIR

OUTPUT_DIR = PROJECT_DIR / 'data/searches/01_cybersec'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)


## Gateway to Research search

In [ ]:
GTR = gtr.GtrGetter(vector_db_path=VECTOR_DB_DIR)

In [ ]:
Search = search.SearchDataset(GTR, GTR.projects_enriched, "config.yaml")

In [ ]:
search_df = Search.do_search()

In [ ]:
search_df.head(50)[['title', 'url', '_score_keywords', '_score_vectors', '_score_avg']]


In [6]:
search_df.to_csv(OUTPUT_DIR / 'gtr_search_results.csv', index=False)

### Check organisations and people

Note, we can select only the-most-likely-to-be-relevant projects by thresholding the score and project start dates.

In [ ]:
top_organisations_df = GTR.get_aggregated_organisations((
    search_df
    .query('_score_avg > 0.3')
    .query('start > "2015"')
))
top_organisations_df.head(15)

In [ ]:
top_persons_df = GTR.get_aggregated_persons((
    search_df
    .query('_score_avg > 0.3')
    .query('start > "2015"')
))
top_persons_df.head(15)

You can also export the people and organisations relevant to each project

In [ ]:
GTR.get_project_stakeholders(search_df)

## Crunchbase search

In [ ]:
CB = crunchbase.CrunchbaseGetter(vector_db_path=VECTOR_DB_DIR)

In [ ]:
SearchCB = search.SearchDataset(CB, CB.organisations_enriched, "config.yaml")

In [ ]:
search_cb_df = SearchCB.do_search()

In [ ]:
search_cb_df.head(50)[['name', 'short_description', 'homepage_url', '_score_keywords', '_score_vectors', '_score_avg']]


In [14]:
search_cb_df.to_csv(OUTPUT_DIR / 'cruchbase_search_results.csv', index=False)

### Check relevant people

In [ ]:
# Produce a table with people for all selected organisations
CB.get_organisations_people(search_cb_df).head(25)

In [ ]:
CB.get_aggregated_people(search_cb_df.query('_score_avg > 0.3')).head(25)